# LUCA Logic
Luca is shaped by multiple objects that will define parameters, and equations to build a pint registery for unit conversion.

Inputs can be provided via a python script or a yaml file. Inputs will include the units to be converted, conversion equation, and input parameters

## Input via python script

In [1]:
from luca import Converter,Parameter,Equation

In [2]:
help(Parameter)

Help on class Parameter in module luca.core:

class Parameter(builtins.object)
 |  Parameter(name: str, abbr: str, unit: str, value: float = None, source: str = None)
 |
 |  A Parameter object that can be constructed with or without a value assigned to it.
 |
 |  Methods defined here:
 |
 |  __init__(self, name: str, abbr: str, unit: str, value: float = None, source: str = None)
 |      _summary_
 |
 |      Parameters
 |      ----------
 |      name : str
 |          name of the parameter
 |      abbr : str
 |          abbreviation of parameter that is corresponding to the equation naming
 |      unit : str
 |          unit of measure
 |      value : float, optional
 |          value of the parameter for a specific item, when assigned, by default None
 |      source : str, optional
 |          source of data when the value is assigned, by default None
 |
 |  __repr__(self)
 |      Return repr(self).
 |
 |  __str__(self)
 |      Return str(self).
 |
 |  get_str(self, item: str)
 |      

In [3]:
help(Equation)

Help on class Equation in module luca.core:

class Equation(builtins.object)
 |  Equation(lh: str, rh: str, eq: str, reverse=True, switched_on=True)
 |
 |  Generates a general equation that can be duplicated for different items for a specific case
 |
 |  Methods defined here:
 |
 |  __init__(self, lh: str, rh: str, eq: str, reverse=True, switched_on=True)
 |      Initializing an Equation class
 |
 |      Parameters
 |      ----------
 |      lh : str
 |          left hand side of the context equation
 |      rh : str
 |          right hand side of the context equation
 |      eq : str
 |          parametric equation
 |      reverse : boolean
 |          if True, the reverse equation will be checked and added automatically
 |
 |      Raises
 |      ------
 |      ValueError
 |          string 'value' should be present for the pint registery
 |
 |      .. code-block:: python
 |
 |          equation = Equation('mass','energy','value * LHV_{gas}')
 |          print(equation)
 |
 |         

In [4]:
help(Converter)

Help on class Converter in module luca.core:

class Converter(builtins.object)
 |  Converter(item_type, equations: list[luca.core.Equation], parameters: list[luca.core.Parameter])
 |
 |  Creates the pint Registery format for an item with collecting Equations, Parameters and assigned values
 |
 |  Methods defined here:
 |
 |  __init__(self, item_type, equations: list[luca.core.Equation], parameters: list[luca.core.Parameter])
 |      Initialize a Converter object
 |
 |      Parameters
 |      ----------
 |      item_type : str
 |          the type of the item for registery, that is correspondant to the parameter place holder in the Equation objects
 |      equations : list[Equation]
 |          a list of all the equations in the registery
 |      parameters : list[Parameter]
 |          a list of all the parameters in the registery
 |
 |  __str__(self)
 |      Return str(self).
 |
 |  copy(self)
 |      Reutrns a copy of the object
 |
 |      Returns
 |      -------
 |      Converter
 |

In [5]:
# Build a parameter
par = Parameter(
    name= "Surface Density",
    abbr="Sden",
    unit = "kg / m**2"
)

In [6]:
print(par)

Sden [kg / m**2]


In [7]:
# now assume we want to calculate area using the mass and surface density of a material. We need to build an equation for the conversion

eq = Equation(
    lh = "area", # what you want to calculate
    rh = "mass", # what you have,
    eq = "value * Sden_{material}" # --> means the value of the rh * (a parameter which is Sden parameter in this case. Note that _{material} is essential to identify different values for different materials)
)

In [8]:
print(eq)

   [area] -> [mass]: value * Sden_{material}


In [9]:
materials = Converter(
    item_type= "material",
    equations= [eq],
    parameters=[par]
    )

In [10]:
materials

In [11]:
print(materials)

the context could not be initialized due to this errors: 
    . Item is not initialized using the specific case.
   . Parameter 'Sden [kg / m**2]' is not assigned with a value.




In [ ]:
materials.is_ready(

)

In [12]:
materials.initialize(
    item_name= "Steel",
    item_abbr="Steel",
    parameters= dict(Sden=0.48)
)

In [13]:
materials.is_ready(

)

True

In [14]:
print(materials)

# Steel 
Sden_Steel = 0.48 kg / m**2
@context Steel = Steel 
   [area] -> [mass]: value * Sden_Steel
   [mass] -> [area]: value / Sden_Steel
@end


In [22]:
materials.to_txt("pint_registry.txt")

If there are multiple registries, you can do the registry in one single shot, by using the Collected object

You can do everything via yaml files in one shot. Look at how yaml files should look like.

In [15]:
from luca.io import parse_context

test = parse_context("luca/yaml_test.yaml")

In [16]:
print(test)

{'gas': <luca.core.Converter object at 0x113734fb0>, 'transportation': <luca.core.Converter object at 0x10dc6a030>}


In [17]:
print(test["gas"])

the context could not be initialized due to this errors: 
    . Item is not initialized using the specific case.
   . Parameter 'LHV [MJ / kg]' is not assigned with a value.
   . Parameter 'dens [kg / m**3]' is not assigned with a value.




In [18]:
print(test["transportation"])

the context could not be initialized due to this errors: 
    . Item is not initialized using the specific case.
   . Parameter 'Price [EUR / tonKM]' is not assigned with a value.




In [20]:
gas = test["gas"]
eqs = gas.equations

In [ ]:
eqs

In [ ]:
gas.get_parameter("LHV")

In [21]:
gas.to_frame({"Natural Gas":"NG","Hydrogen":"H2"})

,Parameter,ParAbbreviation,Unit,item_type,item_name,item_abbr,value,source
0,Lower Heating Value,LHV,MJ / kg,NaN,Natural Gas,NG,NaN,NaN
1,Density,dens,kg / m**3,NaN,Natural Gas,NG,NaN,NaN
0,Lower Heating Value,LHV,MJ / kg,NaN,Hydrogen,H2,NaN,NaN
1,Density,dens,kg / m**3,NaN,Hydrogen,H2,NaN,NaN


In [22]:
gas = gas.parse_from_file("test.xlsx")

In [23]:
gas

# Natrual Gas 
LHV_NG = 7 MJ / kg
dens_NG = 8 kg / m**3
@context Natrual Gas = NG 
   [mass] -> [energy]: value * LHV_NG
   [energy] -> [mass]: value / LHV_NG
   [energy] -> [mass]: value / LHV_NG
   [mass] -> [energy]: value * LHV_NG
@end
# Hydrogen 
LHV_H2 = 7 MJ / kg
dens_H2 = 8 kg / m**3
@context Hydrogen = H2 
   [mass] -> [energy]: value * LHV_H2
   [energy] -> [mass]: value / LHV_H2
   [energy] -> [mass]: value / LHV_H2
   [mass] -> [energy]: value * LHV_H2
@end
# Nitrogen 
LHV_N = 7 MJ / kg
dens_N = 8 kg / m**3
@context Nitrogen = N 
   [mass] -> [energy]: value * LHV_N
   [energy] -> [mass]: value / LHV_N
   [energy] -> [mass]: value / LHV_N
   [mass] -> [energy]: value * LHV_N
@end

In [24]:
pinter = gas.to_pint("gases.txt")

In [25]:
pinter

In [ ]:
pinter.LHV_NG

In [ ]:
pinter("kton")

In [ ]:
print(gas)

In [60]:
distance = 24.0 * pinter.meter


In [ ]:
distance

In [ ]:
time = 8.0 * pinter.second
time

In [63]:
speed = distance/time

In [ ]:
speed

In [ ]:
speed.to(pinter.inch / pinter.minute )

In [ ]:
(10 * pinter.LHV_H2).to()

In [72]:
x = 10*pinter.LHV_H2

In [ ]:
x.magnitude

In [ ]:
x.units

In [ ]:
x.dimensionality

In [ ]:
x.to(pinter.kWh)

In [ ]:
pinter.LHV_H2.dimensionality

In [ ]:
pinter.kWh.dimensionality

In [ ]:
import pint
ureg = pint.UnitRegistry()

ureg.load_definitions('Output/LUCA for pint.txt')


In [ ]:
ureg('truckload').dimensionality

In [ ]:
ureg.Sden_rail_trans

In [ ]:
ureg.LHV_NG

In [ ]:
10 * ureg.LHV_H2

In [ ]:
ureg.truckload.dimensionality

q = ureg.Quantity("2 truckloads")
q.to("kg")

q.to("kg", "Euro_TIR")

q.to("kg", "British_grocer")


In [117]:
q = ureg.Quantity("2 truckloads")

In [ ]:
q

In [ ]:
q.to("kg")

In [ ]:
q.to("kg", "Euro_TIR")

In [121]:
q = 2 * ureg.LHV_H2

In [ ]:
q.to("mass","energy")

In [ ]:
ureg.british_thermal_unit.dimensionality